In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!pip install gdown
!gdown https://drive.google.com/uc?id=12g2h0laV6YZWrM8nWx8QIHdanM_gFYr9

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://se

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


--2021-08-17 23:59:56--  https://users.dcc.uchile.cl/~mtoro/cursos/BD2020/adult_with_pii.csv
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4910401 (4.7M) [text/csv]
Saving to: ‘adult_with_pii.csv’

adult_with_pii.csv  100%[===================>]   4.68M  1.01MB/s    in 4.8s    

2021-08-18 00:00:02 (993 KB/s) - ‘adult_with_pii.csv’ saved [4910401/4910401]



In [ ]:
%%sql
DROP TABLE IF EXISTS person;
CREATE TABLE person
(
  name varchar(255),
  dob date,
  ssn  varchar(50),
  zip integer,
  age integer,
  workclass varchar(255),
  fnlwgt integer,
  education varchar(255),
  education_num integer,
  marital_status varchar(255),
  occupation varchar(255),
  relationship varchar(255),
  race varchar(255),
  sex varchar(50),
  capital_gain integer,
  capital_loss integer,
  hours_per_week integer,
  country varchar(255),
  target varchar(255)
);
ALTER TABLE ONLY person ADD CONSTRAINT "person_key" PRIMARY KEY (name,ssn);

 * postgresql+psycopg2://@/postgres


""


Suponga que el gobierno de Estados Unidos cuenta con un base de datos de resultados del censo (la información de identificación personal (PII) es inventada)

In [ ]:
%%sql
copy person(name, dob, ssn, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target) from
'/content/adult_with_pii.csv' delimiter ',' csv header;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM person limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga que los identificadores explícitos de las personas son las columnas name y ssn (social security number). 
Para proteger la privacidad de las personas, el gobierno de-identifica esta tabla borrando las columnas name y ssn.

In [ ]:
%%sql 
create view person_deid as select dob, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target from person; 

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from person_deid limit 5;

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,1967-09-07,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1988-06-07,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,1991-08-06,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,2009-04-06,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,1951-09-16,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga también que se filtró de una casa comercial, los nombres, fechas de nacimiento, ssn, código postal y edad de muchas personas. Nadie se preocupa por esto ya que no contiene información sensible (como el sueldo).
Crearemos una vista para representar esta data auxiliar.

In [ ]:
%%sql 
create view aux as select name, dob, ssn, zip, age from person; 

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from aux limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38
3,Dorry Poter,2009-04-06,659-57-4974,25503,53
4,Dick Honnan,1951-09-16,220-93-3811,75387,28


#Pregunta 1

Escriba una función que tome como argumento un conjunto de columnas y que realice un ataque de asociación a la tabla person_deid usando la tabla aux.
Muestre el resultado de al menos 3 ataques de asociación.


In [ ]:
def ataque_asociacion(cols):
  condition = ' AND '.join(['a.'+col+'='+'p.'+col for col in cols])
  r = %sql select * from aux a join person_deid p on ($condition)
  return r

In [ ]:
len(ataque_asociacion(['zip']))

 * postgresql+psycopg2://@/postgres


43191

In [ ]:
len(ataque_asociacion(['zip', 'dob']))

 * postgresql+psycopg2://@/postgres


32563

In [ ]:
len(ataque_asociacion(['zip', 'age']))

 * postgresql+psycopg2://@/postgres


32755

#  Pregunta 2

1.   Escriba cuantos individuos en esta base de datos son únicamente identificados por el código postal (zip). ¿A qué porcentaje del total corresponde?
2.   Haga lo mismo para el CId edad (age).
3.   ¿Qué puede concluir de este análisis?




In [ ]:
r = %sql select * from person_deid
total = len(r)

 * postgresql+psycopg2://@/postgres


In [ ]:
r = %sql select name, ssn, count(*) from aux a join person_deid p on (a.zip = p.zip) group by name, ssn having count(*)=1
print(len(r))
len(r)/total*100


 * postgresql+psycopg2://@/postgres
23513


72.21215564632536

In [ ]:
r = %sql select name, ssn, count(*) from aux a join person_deid p on (a.age = p.age) group by name, ssn having count(*)=1
print(len(r))
len(r)/total*100

 * postgresql+psycopg2://@/postgres
2


0.006142317496391388

El campo zip es muy selectivo!

# Pregunta 3
El gobierno se da cuenta de que su intento de anonimización es muy vulnerable, por lo que solo permite a los analistas a realizar consultas de agregación que involucren muchas filas. 
Obtenga el nivel educativo (education_num) del individuo llamado Ardyce Golby usando solo consultas de agregación que involucren al menos 1000 tuplas sobre la tabla *person*.

In [ ]:
big1 = %sql select sum(education_num) from person
big2 = %sql select sum(education_num) from person where name<>'Ardyce Golby'
big1['sum'].iloc[0] - big2['sum'].iloc[0]

 * postgresql+psycopg2://@/postgres
 * postgresql+psycopg2://@/postgres


12

#Pregunta 4


1.   Suponga ahora que un atacante cuenta con data auxiliar que contiene el cuasi-identificador código postal (zip). Indique qué nivel de $k$-Anonimato se cumple la vista person_deid.
2.   Utilice alguna técnica de generalización y/o eliminación de valores atípicos sobre los cuasi-identificadores para lograr al menos $2$-anonimato.
3.   Repita los pasos 1 y 2 para el CId edad (age).
4.   Repita los pasos 1 y 2 para los CIds (zip, age). *Hint*: puede tratar de eliminar valores atípicos para la columna age.
5.   Concluya que es lo que pasa cuando agregamos más CIds al análisis de $k$-anonimato.




In [ ]:
%sql SELECT zip, count(*) c FROM person GROUP BY zip ORDER BY c ASC limit 1;

 * postgresql+psycopg2://@/postgres


,zip,c
0,11233,1


In [ ]:
%sql select (zip/100)*100 as zip, count(*) c from person_deid p group by (zip/100)*100 order by c limit 1

 * postgresql+psycopg2://@/postgres


,zip,c
0,71100,16


In [ ]:
%sql SELECT (age/10)*10, count(*) c FROM person GROUP BY (age/10)*10 ORDER BY c ASC limit 1;

 * postgresql+psycopg2://@/postgres


,?column?,c
0,90,43


In [ ]:
%sql select (zip/1000)*1000 as zip, LEAST((age/10)*10,60) as age, count(*) c from person_deid p group by (zip/1000)*1000, LEAST((age/10)*10,60)  order by c


 * postgresql+psycopg2://@/postgres


,zip,age,c
0,60000,10,5
1,9000,10,8
2,71000,10,10
3,4000,10,10
4,19000,10,11
...,...,...,...
595,79000,20,104
596,61000,30,105
597,50000,30,105
598,97000,30,107


In [ ]:
%sql select zip, (age/10)*10, count(*) c from person_deid p group by zip,(age/10)*10  order by c


 * postgresql+psycopg2://@/postgres


,zip,?column?,c
0,22064,40,1
1,41721,40,1
2,6236,60,1
3,20060,30,1
4,87727,50,1
...,...,...,...
31525,58179,30,3
31526,54853,20,3
31527,6733,40,3
31528,93031,20,3


In [ ]:
%sql select (zip/100)*100 as zip, (age/10)*10 as age, count(*) c from person_deid p group by (zip/100)*100, (age/10)*10 order by c limit 1

UsageError: Line magic function `%sql` not found.


#Pregunta 5

Suponga que los CIds son edad y nivel de educación (age, education_num), y estado civil un dato sensible (marital_status).

1.   Escriba una consulta e indique si la tabla es $2$-diversa.
2.   Utilice generalización y/o eliminación de valores atípicos para lograr al menos $5$-diversidad.
3.   Para el resultado en (2), escriba la consulta e indique el nivel de $k$-anonimato que se logra. ¿Qué puede concluir de esto?
4.   La **diversidad entrópica** de un grupo con el mismo CIds mide cuán uniformemente están distribuidos los valores privados de algún atributo $S$ en el grupo. Se define formalmente como
$$-\sum_{v\in S}\mathtt{P}(\mathit{cIds}, v)*\mathtt{log}(\mathtt{P}(\mathit{cIds}, v))$$ donde $\mathtt{P}(\mathit{cIds}, v)$ corresponde a la fracción de registros en el grupo $\mathit{cIds}$ con valor $v$ para el atributo $S$.
Por ejemplo, para la siguiente tabla donde la enfermedad es un dato sensible:

| Trabajo | Sexo | Edad | Enfermedad |
| --- | --- | --- | --- |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | VIH |
| Artista | Femenino | [35-35) | Resfrío |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |

La diversidad entrópica del grupo (Profesor, Masculino [35-40)) es de $-(\frac{2}{3}\mathtt{log}\frac{2}{3} + \frac{1}{3}\mathtt{log}\frac{1}{3}) = \mathtt{log}(1.9)$, y la diversidad entropica del grupo (Artista, Femenino, [35-35)) es $-(\frac{3}{4}\mathtt{log}\frac{3}{4} + \frac{1}{4}\mathtt{log}\frac{1}{4}) = \mathtt{log}(1.8)$.

Calcule la **diversidad entrópica** de cada grupo en (2), y concluya cuál es el conjunto de cids más y menos diversos.



In [ ]:
%sql select age, education_num, count(distinct marital_status) c from person_deid group by age, education_num order by c limit 1

 * postgresql+psycopg2://@/postgres


,age,education_num,c
0,17,3,1


In [ ]:
%sql select LEAST((age/10)*10,60), (education_num/10)*10, count(distinct marital_status) c from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10 order by c limit 1

 * postgresql+psycopg2://@/postgres


,least,?column?,c
0,10,10,5


In [ ]:
%sql select LEAST((age/10)*10,60), (education_num/10)*10, count(*) c from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10 order by c limit 1

 * postgresql+psycopg2://@/postgres


,least,?column?,c
0,10,10,455


In [ ]:
%%sql
  select a, e, -sum(c1::float/c2*log(c1::float/c2)) as P from
  (select LEAST((age/10)*10,60) as a, (education_num/10)*10 as e, marital_status, count(*) c1 from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10, marital_status) r1
  natural join
  (select LEAST((age/10)*10,60) as a, (education_num/10)*10 as e, count(*) c2 from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10) r2
 group by a,e 
 order by P
  

 * postgresql+psycopg2://@/postgres


,a,e,p
0,10,10,0.032632
1,10,0,0.080227
2,20,10,0.335222
3,20,0,0.444796
4,50,10,0.456344
5,60,10,0.487969
6,40,10,0.488227
7,30,10,0.501664
8,60,0,0.510638
9,50,0,0.516622


In [ ]:
%sql select LEAST((age/10)*10,60) as edad, (education_num/10)*10 as nivel_educativo, marital_status, count(*) c1 from person_deid where LEAST((age/10)*10,60)=10 and (education_num/10)*10=10 group by LEAST((age/10)*10,60), (education_num/10)*10, marital_status 

 * postgresql+psycopg2://@/postgres


,edad,nivel_educativo,marital_status,c1
0,10,10,Divorced,2
1,10,10,Married-civ-spouse,1
2,10,10,Married-spouse-absent,1
3,10,10,Never-married,450
4,10,10,Separated,1


In [ ]:
%sql select LEAST((age/10)*10,60) as a, (education_num/10)*10 as e, count(*) c from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10

 * postgresql+psycopg2://@/postgres


,a,e,c
0,20,0,3380
1,50,10,2246
2,30,0,3733
3,50,0,2172
4,30,10,4880
5,10,0,1202
6,40,10,4380
7,60,10,1172
8,20,10,4674
9,40,0,2795


In [ ]:
%%sql 
select r1.a, r1.e, -sum(r1.c::decimal/r2.c*log(r1.c::decimal/r2.c)) as entropia from
  (select LEAST((age/10)*10,60) as a, (education_num/10)*10 as e, marital_status, count(*) c1 from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10, marital_status) r1
  natural join
  (select LEAST((age/10)*10,60) as a, (education_num/10)*10 as e, count(*) c2 from person_deid group by LEAST((age/10)*10,60), (education_num/10)*10) r2
  on (r1.a = r2.a and r1.e = r2.e)
group by r1.a, r1.e
order by entropia


 * postgresql+psycopg2://@/postgres


,a,e,entropia
0,10,10,0.0326318555796780791672399175941520797607
1,10,0,0.0802272028943103363810891534086469037957
2,20,10,0.3352221605431637409447062880431015664586
3,20,0,0.4447959357538507484924126578534361503652
4,50,10,0.4563439027361248960066197283688486581562
5,60,10,0.4879692405900158268107783502526202397094
6,40,10,0.4882265716852997149660377017675083481434
7,30,10,0.5016643048932219344506556104040466186334
8,60,0,0.5106382313591667395424406247049378911264
9,50,0,0.5166219415987672948463339986454721740488


# Pregunta 6

1.   Escriba una consulta que responda a "¿cuantas personas nunca se han casado?"
2.   La sensibilidad de una consulta se puede calcular como: cuánto es lo máximo que puede variar el resultado de mi consulta si saco/agrego un único individuo de la base de datos. Escriba y justifique la sensibilidad de la consulta de (1).
3.   Use la distribución de laplace para contestar (1) de manera diferencialmente privada, con epsilon = 0.1. 
    
    *Hint*: Para generar ruido aleatorio puede utilizar la función laplace de numpy:
```
import numpy as np
np.random.laplace(loc=0, scale=s)
```
y para aplicar una función a todos los elementos de un resultado de un consulta (dataframe) puede usar la función apply:
```
r = %sql ...
r.apply(lambda x: ...)
```
4.   Escriba una consulta $0.1$-diferencialmente privada que calcule la cantidad de personas agrupadas por edad y estado civil. Asegúrese que su respuesta no contenga cantidades negativas ni decimales (puede modificar los valores después de agregarle ruido).

    *Hint*: Use la función map para aplicar una función a todos los elementos de una columna de la siguiente forma:
```
r = %sql ...
r['nombre_columna_nueva'] = r['nombre_columna'].map(lambda x: ...)
```

5.    Comente acerca de la utilidad y la privadcidad de la respuesta para grupos muy chicos vs grupos grandes después de agregar ruido. 


In [ ]:
%sql select count(*) from person where marital_status='Never-married'

 * postgresql+psycopg2://@/postgres


,count
0,10683


La sensibilidad es 1. Si agrego una persona, el resultado puede cambiar a lo más en 1.

In [ ]:
import numpy as np
r = %sql select count(*) from person where marital_status='Never-married'
r.apply(lambda x: x + np.random.laplace(loc=0, scale=1/0.1))

 * postgresql+psycopg2://@/postgres


,count
0,10676.396483


In [ ]:
r = %sql select age, marital_status, count(*) from person group by age, marital_status order by age, marital_status
#r.apply(lambda x: x + np.random.laplace(loc=0, scale=1/0.1))
r['countdp'] = r['count'].map(lambda x: round(max(0,x + np.random.laplace(loc=0, scale=1/0.1))))
r


 * postgresql+psycopg2://@/postgres


,age,marital_status,count,countdp
0,17,Married-civ-spouse,2,21
1,17,Never-married,393,390
2,18,Divorced,1,18
3,18,Married-civ-spouse,7,29
4,18,Married-spouse-absent,1,3
...,...,...,...,...
391,90,Divorced,1,0
392,90,Married-civ-spouse,20,29
393,90,Never-married,14,0
394,90,Separated,2,0


Para grupos muy chicos las respuestas son poco útiles ya que el ruido distorciona mucho la respuesta real. Pero esto esta bien desde el punto de privacidad ya que los grupos muy chicos son muy selectivos y por ende tienen una gran probabilidad de violar privacidad.

In [ ]:
def foo(tabla):
  r = %sql select * from $tabla
  return r
foo('person')